In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 분실물 매칭 시스템 테스트 노트북\n",
    "\n",
    "이 노트북은 PySpark를 이용한 임베딩 유사도 비교 시스템을 테스트합니다. 하둡에 저장된 임베딩과 새 분실물 이미지 간의 유사도를 계산하고 결과를 시각화합니다."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# 필요한 모듈 임포트\n",
    "import os\n",
    "import sys\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from PIL import Image\n",
    "import torch\n",
    "import time\n",
    "from pyspark.sql import SparkSession\n",
    "\n",
    "# 현재 디렉토리를 파이썬 경로에 추가\n",
    "module_path = os.path.abspath(os.path.join('..'))\n",
    "if module_path not in sys.path:\n",
    "    sys.path.append(module_path)\n",
    "\n",
    "# 자체 모듈 임포트\n",
    "from src.similarity_engine import SimilarityEngine\n",
    "from src.utils import setup_logging, create_image_embedding, load_clip_model, format_results"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. PySpark 세션 초기화"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Spark 세션 생성\n",
    "spark = SparkSession.builder \\\n",
    "    .appName(\"LostFoundSimilarityTest\") \\\n",
    "    .config(\"spark.executor.memory\", \"4g\") \\\n",
    "    .config(\"spark.driver.memory\", \"2g\") \\\n",
    "    .config(\"spark.python.profile\", \"true\") \\\n",
    "    .getOrCreate()\n",
    "\n",
    "print(f\"Spark 버전: {spark.version}\")\n",
    "print(f\"Spark 설정: \\n{spark.sparkContext.getConf().getAll()}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. 로깅 설정 및 환경 확인"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# 로깅 설정\n",
    "logger = setup_logging(log_dir='./logs')\n",
    "logger.info(\"분실물 매칭 시스템 테스트 시작\")\n",
    "\n",
    "# HDFS 환경 변수 확인\n",
    "hdfs_conf = {}\n",
    "hdfs_conf['fs.defaultFS'] = os.environ.get('HDFS_NAMENODE_ADDRESS', 'hdfs://localhost:9000')\n",
    "print(f\"HDFS 구성: {hdfs_conf}\")\n",
    "\n",
    "# 하둡 임베딩 경로 설정\n",
    "hadoop_embeddings_path = os.environ.get('HADOOP_EMBEDDINGS_PATH', '/user/lost_found/embeddings')\n",
    "print(f\"임베딩 경로: {hadoop_embeddings_path}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. CLIP 모델 로드 (임베딩 확인용)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# CLIP 모델 로드\n",
    "model, processor = load_clip_model()\n",
    "\n",
    "if model is not None and processor is not None:\n",
    "    logger.info(\"CLIP 모델 로드 성공\")\n",
    "    print(\"CLIP 모델 정보:\")\n",
    "    print(f\"- 모델 타입: {type(model).__name__}\")\n",
    "    print(f\"- 모델 임베딩 차원: {model.config.projection_dim}\")\n",
    "else:\n",
    "    logger.error(\"CLIP 모델 로드 실패\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. 유사도 엔진 초기화"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# 유사도 엔진 초기화\n",
    "similarity_engine = SimilarityEngine(\n",
    "    hadoop_embeddings_path=hadoop_embeddings_path,\n",
    "    threshold=0.7  # 70% 유사도 임계값\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. 하둡에서 임베딩 데이터 로드 및 확인"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# 임베딩 데이터 로드\n",
    "embeddings_df = similarity_engine.load_embeddings()\n",
    "\n",
    "if embeddings_df is not None:\n",
    "    # 데이터프레임 구조 확인\n",
    "    print(\"임베딩 데이터 스키마:\")\n",
    "    embeddings_df.printSchema()\n",
    "    \n",
    "    # 데이터 샘플 확인\n",
    "    print(\"\\n임베딩 데이터 샘플 (5개):\")\n",
    "    embeddings_df.select(\"id\", \"title\", \"created_at\").show(5, truncate=False)\n",
    "    \n",
    "    # 임베딩 통계 확인\n",
    "    emb_count = embeddings_df.count()\n",
    "    print(f\"\\n총 임베딩 개수: {emb_count}\")\n",
    "    \n",
    "    # 임베딩 벡터 확인 (첫 번째 행)\n",
    "    first_emb = embeddings_df.select(\"embedding\").first()\n",
    "    if first_emb:\n",
    "        emb_array = np.array(first_emb[0])\n",
    "        print(f\"임베딩 벡터 형태: {emb_array.shape}\")\n",
    "        print(f\"벡터 처음 5개 값: {emb_array[:5]}\")\n",
    "else:\n",
    "    print(\"임베딩 데이터를 로드할 수 없습니다.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. 테스트 이미지 로드 및 임베딩 생성"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# 테스트 이미지 경로 (분실물 신고 이미지)\n",
    "test_image_path = \"../test_data/sample_lost_item.jpg\"\n",
    "\n",
    "# 이미지 표시\n",
    "try:\n",
    "    img = Image.open(test_image_path)\n",
    "    plt.figure(figsize=(6, 6))\n",
    "    plt.imshow(img)\n",
    "    plt.axis('off')\n",
    "    plt.title('테스트 이미지 (분실물)')\n",
    "    plt.show()\n",
    "except Exception as e:\n",
    "    logger.error(f\"이미지 로드 실패: {str(e)}\")\n",
    "    print(f\"이미지를 로드할 수 없습니다: {str(e)}\")\n",
    "\n",
    "# 이미지 임베딩 생성\n",
    "query_embedding = create_image_embedding(test_image_path, model, processor)\n",
    "\n",
    "if query_embedding is not None:\n",
    "    logger.info(f\"이미지 임베딩 생성 성공: 형태 {query_embedding.shape}\")\n",
    "    print(f\"임베딩 생성 완료: 차원 {query_embedding.shape}\")\n",
    "    \n",
    "    # 임베딩 시각화 (처음 100개 차원)\n",
    "    plt.figure(figsize=(10, 4))\n",
    "    plt.plot(query_embedding[:100])\n",
    "    plt.title('임베딩 벡터 시각화 (처음 100개 차원)')\n",
    "    plt.grid(True)\n",
    "    plt.show()\n",
    "else:\n",
    "    logger.error(\"이미지 임베딩 생성 실패\")\n",
    "    print(\"임베딩을 생성할 수 없습니다.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. 유사도 계산 및 결과 추출"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# 유사도 임계값 설정\n",
    "threshold = 0.65  # 65% 유사도\n",
    "top_k = 10       # 상위 10개 결과\n",
    "\n",
    "# 실행 시간 측정\n",
    "start_time = time.time()\n",
    "\n",
    "# 분실물 이미지와 유사한 아이템 찾기\n",
    "similar_items = similarity_engine.find_similar_items(\n",
    "    query_embedding=query_embedding,\n",
    "    threshold=threshold,\n",
    "    top_k=top_k\n",
    ")\n",
    "\n",
    "end_time = time.time()\n",
    "duration = end_time - start_time\n",
    "\n",
    "print(f\"유사도 계산 완료! (소요 시간: {duration:.2f}초)\")\n",
    "print(f\"유사한 아이템 수: {len(similar_items)}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. 결과 분석 및 시각화"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# 결과가 있는 경우 분석\n",
    "if similar_items:\n",
    "    # 데이터프레임으로 변환\n",
    "    results_df = pd.DataFrame(similar_items)\n",
    "    \n",
    "    # 결과 출력\n",
    "    print(\"유사한 아이템 목록:\")\n",
    "    display(results_df)\n",
    "    \n",
    "    # 유사도 분포 시각화\n",
    "    plt.figure(figsize=(10, 6))\n",
    "    sns.barplot(x='id', y='similarity', data=results_df)\n",
    "    plt.title(f'유사도 분포 (임계값: {threshold*100}%)')\n",
    "    plt.xlabel('아이템 ID')\n",
    "    plt.ylabel('유사도 (%)')\n",
    "    plt.xticks(rotation=45)\n",
    "    plt.grid(axis='y')\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    # 상위 3개 아이템 표시\n",
    "    print(\"\\n상위 3개 유사 아이템:\")\n",
    "    for i, item in enumerate(similar_items[:3], 1):\n",
    "        print(f\"{i}. {item['title']} - 유사도: {item['similarity']}%\")\n",
    "        \n",
    "    # 결과 저장 (선택사항)\n",
    "    # results_df.to_csv(\"similarity_results.csv\", index=False)\n",
    "else:\n",
    "    print(f\"임계값 {threshold*100}%를 충족하는 유사한 아이템이 없습니다.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. 임계값 조정 및 재실행 테스트"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# 낮은 임계값으로 다시 시도\n",
    "lower_threshold = 0.5  # 50% 유사도\n",
    "\n",
    "print(f\"임계값을 {lower_threshold*100}%로 낮추어 다시 시도합니다...\")\n",
    "\n",
    "# 재실행\n",
    "similar_items_lower = similarity_engine.find_similar_items(\n",
    "    query_embedding=query_embedding,\n",
    "    threshold=lower_threshold,\n",
    "    top_k=top_k\n",
    ")\n",
    "\n",
    "print(f\"유사한 아이템 수 (임계값 {lower_threshold*100}%): {len(similar_items_lower)}\")\n",
    "\n",
    "# 결과가 있는 경우 분석\n",
    "if similar_items_lower:\n",
    "    # 데이터프레임으로 변환\n",
    "    results_df_lower = pd.DataFrame(similar_items_lower)\n",
    "    \n",
    "    # 결과 출력\n",
    "    print(\"\\n유사한 아이템 목록 (낮은 임계값):\")\n",
    "    display(results_df_lower)\n",
    "    \n",
    "    # 유사도 분포 시각화\n",
    "    plt.figure(figsize=(12, 6))\n",
    "    sns.barplot(x='id', y='similarity', data=results_df_lower)\n",
    "    plt.axhline(y=threshold*100, color='r', linestyle='--', label=f'원래 임계값 ({threshold*100}%)')\n",
    "    plt.title(f'유사도 분포 (임계값: {lower_threshold*100}%)')\n",
    "    plt.xlabel('아이템 ID')\n",
    "    plt.ylabel('유사도 (%)')\n",
    "    plt.xticks(rotation=45)\n",
    "    plt.grid(axis='y')\n",
    "    plt.legend()\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "else:\n",
    "    print(f\"임계값 {lower_threshold*100}%를 충족하는 유사한 아이템이 없습니다.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 10. 성능 분석 및 최적화 테스트"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# 다양한 임계값에 대한 성능 테스트\n",
    "thresholds = [0.9, 0.8, 0.7, 0.6, 0.5]\n",
    "results = []\n",
    "\n",
    "for t in thresholds:\n",
    "    # 시작 시간\n",
    "    start = time.time()\n",
    "    \n",
    "    # 유사도 계산\n",
    "    items = similarity_engine.find_similar_items(\n",
    "        query_embedding=query_embedding,\n",
    "        threshold=t,\n",
    "        top_k=top_k\n",
    "    )\n",
    "    \n",
    "    # 종료 시간\n",
    "    end = time.time()\n",
    "    \n",
    "    # 결과 저장\n",
    "    results.append({\n",
    "        'threshold': t*100,  # 백분율로 변환\n",
    "        'item_count': len(items),\n",
    "        'execution_time': end - start\n",
    "    })\n",
    "\n",
    "# 데이터프레임으로 변환\n",
    "perf_df = pd.DataFrame(results)\n",
    "display(perf_df)\n",
    "\n",
    "# 성능 시각화\n",
    "fig, ax1 = plt.subplots(figsize=(10, 6))\n",
    "\n",
    "# 임계값 대 아이템 수\n",
    "ax1.set_xlabel('임계값 (%)')\n",
    "ax1.set_ylabel('매칭 아이템 수', color='tab:blue')\n",
    "ax1.plot(perf_df['threshold'], perf_df['item_count'], 'o-', color='tab:blue')\n",
    "ax1.tick_params(axis='y', labelcolor='tab:blue')\n",
    "\n",
    "# 임계값 대 실행 시간\n",
    "ax2 = ax1.twinx()\n",
    "ax2.set_ylabel('실행 시간 (초)', color='tab:red')\n",
    "ax2.plot(perf_df['threshold'], perf_df['execution_time'], 'o-', color='tab:red')\n",
    "ax2.tick_params(axis='y', labelcolor='tab:red')\n",
    "\n",
    "plt.title('임계값에 따른 성능 분석')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 11. 자원 정리"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Spark 세션 종료\n",
    "similarity_engine.stop()\n",
    "logger.info(\"유사도 엔진 종료\")\n",
    "print(\"테스트 완료!\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}